In [7]:
import numpy as np
import pandas as pd
from mnist import load_mnist

from daml.metrics.drift import CVMDrift, KSDrift, MMDDrift

In [8]:
# Create
rng = np.random.default_rng(33)
size = 1000

test_images, labels = load_mnist(size)
test_images = test_images / 255

rng.shuffle(test_images)
print(test_images.shape)

(1000, 28, 28)


In [9]:
ks = KSDrift(test_images)
cvm = CVMDrift(test_images)
mmd = MMDDrift(test_images)

In [10]:
ksr = ks.predict(test_images * 1.01)
cvmr = cvm.predict(test_images * 1.01)
mmdr = mmd.predict(test_images * 1.01)

any_drift = mmdr["is_drift"] or cvmr["is_drift"] or ksr["is_drift"]
print(any_drift)

drift_preds = {"Maximum Mean Discrepency": mmdr, "Cramér-von Mises": cvmr, "Kolmogorov-Smirnov": ksr}

/workspaces/daml/src/daml/_internal/metrics/drift/ks.py:99: RuntimeWarning: ks_2samp: Exact calculation unsuccessful. Switching to method=asymp.
  dist[f], p_val[f] = ks_2samp(x_ref[:, f], x[:, f], alternative=self.alternative, method="exact")


1


In [13]:
from gradient.slide_deck.shapes import SubText, Table, Text, TextContent
from gradient.slide_deck.slidedeck import DefaultGradientSlideLayouts


def generate_drift_rollup(drift_preds: dict) -> bool:
    return any(preds["is_drift"] for preds in drift_preds.values())


def generate_drift_report_slide_kwargs(drift_preds: dict) -> dict:
    text = [
        "Operational dataset ",
        SubText(f"{'has' if any_drift else 'has not'}", bold=True),
        " drifted from Development dataset",
    ]

    drift_table = pd.DataFrame(
        {
            "Method": list(drift_preds),
            "Has drifted?": ["Yes" if preds["is_drift"] else "No" for preds in drift_preds.values()],
            "Test statistic": [np.mean(preds["distance"]) for preds in drift_preds.values()],
            "Threshold": [preds["threshold"] for preds in drift_preds.values()],
            "Significance": [np.mean(preds["p_val"]) for preds in drift_preds.values()],
        }
    )

    kwargs = {
        "title": "Drift Detection: Summary",
        "layout": DefaultGradientSlideLayouts.CONTENT_DEFAULT,
        "placeholder_fillings": [TextContent(lines=[Text(content=text)])],
        "additional_shapes": [
            Table(
                dataframe=drift_table,
                fontsize=16,
                left=2.0,
                top=2.0,
                width=9.0,
                height=4.0,
            ),
        ],
    }
    return kwargs

In [14]:
generate_drift_rollup(drift_preds)

True

In [ ]:
from pathlib import Path

from gradient.slide_deck.slidedeck import DEFAULT_GRADIENT_PRESENTATION_TEMPLATE_PATH, SlideDeck

example_directory = Path.cwd() / "report_drift_example"
example_directory.mkdir(parents=True, exist_ok=True)

deck = SlideDeck(presentation_template_path=DEFAULT_GRADIENT_PRESENTATION_TEMPLATE_PATH)

deck.add_slide(**generate_drift_report_slide_kwargs(drift_preds))

deck.save(
    output_directory=example_directory,
    name="report_drift_example",
)